In [3]:
"""
Written by, 
Sriram Ravindran, sriram@ucsd.edu

Original paper - https://arxiv.org/abs/1611.08024

Please reach out to me if you spot an error.
"""

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

In [25]:
# load the dataframe from the pickle file
import pickle
dir = "C:/Users/gusta/OneDrive/Skrivebord/KI & Data/Bachelor/LegeData"
with open(f"{dir}/dataframe.pkl", "rb") as f:
    df = pickle.load(f)

# keep these channels only, these are the indexes: 
""" 
Fp1 -> 0
Fp2 -> 33
F3  -> 4
F4  -> 38
C3  -> 12
C4  -> 48
P3  -> 20
P4  -> 55
O1  -> 26
O2  -> 61
F7  -> 6
F8  -> 40
T7  -> 14
T8  -> 50
P7  -> 22
P8  -> 57
Fz  -> 36
Cz  -> 46
Pz  -> 30 

but add 1 to each index, since the first channel is channel_1
"""

df = df[["channel_1", "channel_34", "channel_5", "channel_39", "channel_13", "channel_49", "channel_21", "channel_56", "channel_27", "channel_62", "channel_7", "channel_41", "channel_15", "channel_51", "channel_23", "channel_58", "channel_37", "channel_47", "channel_31", "label"]]
df = df.T
df.head()
# print(df.iloc[:, :3])





,0,1,2,3,4,5,6,7,8,9,...,762870,762871,762872,762873,762874,762875,762876,762877,762878,762879
channel_1,1.412883e+07,1.410380e+07,1.410851e+07,1.411343e+07,-4.909065e+06,-4.948905e+06,-4.949609e+06,-4.952393e+06,3.253510e+06,3.231366e+06,...,-5.643658e+06,-5.641706e+06,-3.462086e+06,-3.462502e+06,-3.461254e+06,-3.461862e+06,-5.158634e+06,-5.159946e+06,-5.155018e+06,-5.154762e+06
channel_34,1.411228e+07,1.410508e+07,1.411683e+07,1.411913e+07,-4.927497e+06,-4.954985e+06,-4.941001e+06,-4.947593e+06,3.241126e+06,3.229286e+06,...,-5.643402e+06,-5.647914e+06,-3.457190e+06,-3.457830e+06,-3.459910e+06,-3.461766e+06,-5.158218e+06,-5.154698e+06,-5.154378e+06,-5.156010e+06
channel_5,1.412431e+07,1.408847e+07,1.411033e+07,1.411014e+07,-4.917001e+06,-4.967753e+06,-4.954601e+06,-4.961673e+06,3.247974e+06,3.218182e+06,...,-5.644170e+06,-5.642218e+06,-3.459174e+06,-3.458790e+06,-3.462886e+06,-3.461894e+06,-5.157930e+06,-5.153930e+06,-5.157130e+06,-5.158090e+06
channel_39,1.410553e+07,1.411987e+07,1.410783e+07,1.411574e+07,-4.932649e+06,-4.927177e+06,-4.953513e+06,-4.954217e+06,3.228486e+06,3.243974e+06,...,-5.645386e+06,-5.643914e+06,-3.459238e+06,-3.460870e+06,-3.459878e+06,-3.461062e+06,-5.156106e+06,-5.157866e+06,-5.154922e+06,-5.156394e+06
channel_13,1.412159e+07,1.410703e+07,1.409171e+07,1.411798e+07,-4.914921e+06,-4.943785e+06,-4.961129e+06,-4.948905e+06,3.250566e+06,3.230438e+06,...,-5.647434e+06,-5.641610e+06,-3.461734e+06,-3.461158e+06,-3.460966e+06,-3.461670e+06,-5.155690e+06,-5.155498e+06,-5.158602e+06,-5.156458e+06


In [3]:
labels = []
for _ in range(10):
    labels.append(np.repeat([1, 0], 149))
labels = np.concatenate(labels)


In [17]:
from torch.utils.data import DataLoader, Dataset

class EEGDataset(Dataset):
    def __init__(self, df, labels):
        self.df = df
        self.labels = labels
        assert len(df.T) == len(labels) * 256, "Mismatch between df rows and expected length from labels"

    def __len__(self):
        return len(self.df.T) // 256  # Ensure we have complete batches

    def __getitem__(self, idx):
        start_idx = idx * 256
        end_idx = start_idx + 256
        data = self.df.iloc[:,start_idx:end_idx].values
        label = self.labels[idx]
        return torch.tensor(data).float(), torch.tensor(label).float()

# Check dataset length
dataset = EEGDataset(df, labels)
print(f"Length of dataset: {len(dataset)}")

# Check DataLoader
dataloader = DataLoader(dataset, batch_size=1)

for batch_ndx, (batch, label) in enumerate(dataloader):
    print(f"Batch {batch_ndx + 1}: Data shape {batch.shape}, Label {label}")
    break  # To print just the first batch


Length of dataset: 2980
Batch 1: Data shape torch.Size([1, 19, 256]), Label tensor([1.])


In [12]:
print(len(df.T))
print(len(labels)*256)

762880
762880


In [19]:
channels = 19
sample_len = 256

class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.T = 120
        
        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, channels), padding = 0)
        self.batchnorm1 = nn.BatchNorm2d(16, False)
        
        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        self.conv2 = nn.Conv2d(1, 4, (2, 32))
        self.batchnorm2 = nn.BatchNorm2d(4, False)
        self.pooling2 = nn.MaxPool2d(2, 4)
        
        # Layer 3
        self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv3 = nn.Conv2d(4, 4, (8, 4))
        self.batchnorm3 = nn.BatchNorm2d(4, False)
        self.pooling3 = nn.MaxPool2d((2, 4))
        
        # FC Layer
        # NOTE: This dimension will depend on the number of timestamps per sample in your data.
        # I have 120 timepoints. 
        self.fc1 = nn.Linear(8*2 * (sample_len // 32), 1)
        

    def forward(self, x):
        # Layer 1
        x = F.elu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.25)
        x = x.permute(0, 3, 1, 2)
        
        # Layer 2
        x = self.padding1(x)
        x = F.elu(self.conv2(x))
        x = self.batchnorm2(x)
        x = F.dropout(x, 0.25)
        x = self.pooling2(x)
        
        # Layer 3
        x = self.padding2(x)
        x = F.elu(self.conv3(x))
        x = self.batchnorm3(x)
        x = F.dropout(x, 0.25)
        x = self.pooling3(x)
        
        # FC Layer
        x = x.reshape(-1, 128)
        x = F.sigmoid(self.fc1(x))
        return x


net = EEGNet()
print(net.forward(Variable(torch.Tensor(1,1,sample_len,channels))))
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters())

tensor([[0.1571]], grad_fn=<SigmoidBackward0>)


#### Evaluate function returns values of different criteria like accuracy, precision etc. 
In case you face memory overflow issues, use batch size to control how many samples get evaluated at one time. Use a batch_size that is a factor of length of samples. This ensures that you won't miss any samples.

In [63]:
def evaluate(model, data_loader, params = ["acc"]):
    results = []
    model.eval()
        
    predicted = []
    
    for i, data in enumerate(data_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # make shape (samples, 1, sample_len, channels)
        inputs = inputs.reshape(-1, 1, sample_len, channels)

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)
        
        predicted = model(inputs)
        
        predicted = predicted.data.cpu().numpy()
    
    for param in params:
        if param == 'acc':
            results.append(accuracy_score(labels, np.round(predicted)))
        if param == "auc":
            results.append(roc_auc_score(labels, predicted))
        if param == "recall":
            results.append(recall_score(labels, np.round(predicted)))
        if param == "precision":
            results.append(precision_score(labels, np.round(predicted)))
        if param == "fmeasure":
            precision = precision_score(labels, np.round(predicted))
            recall = recall_score(labels, np.round(predicted))
            results.append(2*precision*recall/ (precision+recall))
    return results

#### Generate random data

##### Data format:
Datatype - float32 (both X and Y) <br>
X.shape - (#samples, 1, #timepoints,  #channels) <br>
Y.shape - (#samples)

In [47]:
channels = 19
sample_len = 256
no_samples = 2980//10

X_train = np.random.rand(no_samples, 1, sample_len, channels).astype('float32') # np.random.rand generates between [0, 1)
y_train = np.round(np.random.rand(no_samples).astype('float32')) # binary data, so we round it to 0 or 1.

X_val = np.random.rand(no_samples, 1, sample_len, channels).astype('float32')
y_val = np.round(np.random.rand(no_samples).astype('float32'))

X_test = np.random.rand(no_samples, 1, sample_len, channels).astype('float32')
y_test = np.round(np.random.rand(no_samples).astype('float32'))

In [59]:
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

# Dataset class definition
class EEGDataset(Dataset):
    def __init__(self, df, labels):
        self.df = df
        self.labels = labels

    def __len__(self):
        return len(self.df.T) // 256  # Ensure we have complete batches

    def __getitem__(self, idx):
        start_idx = idx * 256
        end_idx = start_idx + 256
        data = self.df[:, start_idx:end_idx]
        label = self.labels[idx*256]
        return torch.tensor(data).float(), torch.tensor(label).float()

# Assume df is a DataFrame where last row is labels, others are features
# Transpose the dataframe to align features and labels correctly
X = df.iloc[:-1, :]  # Features
y = df.iloc[-1, :]   # Labels

# Split into training and test sets using sklearn's train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.T, y, test_size=0.2, random_state=42)

# Reset index for both X and y to avoid KeyError
X_train = X_train.reset_index(drop=True).T.values
X_test = X_test.reset_index(drop=True).T.values
y_train = y_train.reset_index(drop=True).values
y_test = y_test.reset_index(drop=True).values


# Create Dataset and DataLoader for train and test
train_dataset = EEGDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)

test_dataset = EEGDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Check DataLoader and print sizes of datasets and labels
for batch_ndx, (batch, label) in enumerate(train_loader):
    print(f"Train Batch {batch_ndx + 1}: Data shape {batch.shape}, Label shape {label.shape}")
    break  # To print just the first batch

print(f"Training set length: {len(train_dataset)}")
print(f"Test set length: {len(test_dataset)}")


Train Batch 1: Data shape torch.Size([1, 19, 256]), Label shape torch.Size([1])
Training set length: 2384
Test set length: 596


#### Run

In [68]:
batch_size = 16
loops = 1

for epoch in range(loops):  # loop over the dataset multiple times
    print("\nEpoch ", epoch)
    net.train()
    
    running_loss = 0.0

    for i, data in enumerate(train_loader, 0):
        #make progress print every 200 samples
        if i % 200 == 0:
            print(i, end=", ")
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # make shape (samples, 1, sample_len, channels)
        inputs = inputs.reshape(-1, 1, sample_len, channels)

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs).reshape(-1)
        loss = criterion(outputs, labels)
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
    
    # Validation accuracy
    params = ["acc"]
    print(params)
    print("Training Loss ", running_loss)
    print("Train - ", evaluate(net, train_loader, params))
    print("Test - ", evaluate(net, test_loader, params))


Epoch  0
0, 200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, ['acc']
Training Loss  1632.6879984140396
Train -  [1.0]
Test -  [0.0]
